In [5]:
import duckdb
import pandas as pd

In [3]:
# Import CSVs -- the larger CSV has encoding issues, so load via Pandas instead of duckdb.  We can query it in any case!
smol_data = duckdb.read_csv('./van_ct.csv')
big_data = pd.read_csv('./bc_ct.csv',encoding='iso8859_15')

In [19]:
# Rename columns with unsuitable characters in their column names
big_data["C2_COUNT_MEN"] = big_data["C2_COUNT_MEN+"]
big_data["C3_COUNT_WOMEN"] = big_data["C3_COUNT_WOMEN+"]

In [21]:
# Filter to columns and rows of interest
big_data_narrow = duckdb.sql("SELECT DGUID, ALT_GEO_CODE, CHARACTERISTIC_ID, CHARACTERISTIC_NAME, C1_COUNT_TOTAL, C2_COUNT_MEN, C3_COUNT_WOMEN FROM big_data")
big_data_vancouver = duckdb.sql("SELECT * FROM big_data_filtered WHERE ALT_GEO_CODE >= 9330000 AND ALT_GEO_CODE < 9340000") # Vancouver DGUIDs start with 933XXXX
big_data_commutes = duckdb.sql("SELECT * FROM big_data_vancouver WHERE CHARACTERISTIC_ID BETWEEN 2603 and 2610") # Commute share charcteristics

In [45]:
# Do some pivots to get characteristics of interest into a "wide" format
total_pivot = duckdb.sql("PIVOT big_data_commutes ON CHARACTERISTIC_ID USING SUM(C1_COUNT_TOTAL) GROUP BY DGUID")
men_pivot = duckdb.sql("PIVOT big_data_commutes ON CHARACTERISTIC_ID USING SUM(C2_COUNT_MEN) GROUP BY DGUID")
women_pivot = duckdb.sql("PIVOT big_data_commutes ON CHARACTERISTIC_ID USING SUM(C3_COUNT_WOMEN) GROUP BY DGUID")

In [58]:
"""
Characteristics of Interest
2603	Total - Main mode of commuting for the employed labour force aged 15 years and over with a usual place of work or no fixed workplace address - 25% sample data (200)
2604	  Car, truck or van
2605	    Car, truck or van - as a driver
2606	    Car, truck or van - as a passenger
2607	  Public transit
2608	  Walked
2609	  Bicycle
2610	  Other method
"""
# too lazy to write out the same renaming query 3 times
renamed = {}
for segment in ['total','men','women']:
    renamed[segment] = duckdb.sql(f'select DGUID, \
               "2603" as {segment}_total, \
               "2604" as {segment}_car, "2605" as {segment}_car_driver, "2606" as {segment}_car_passenger,\
               "2607" as {segment}_public_transit,\
               "2608" as {segment}_walk, "2609" as {segment}_bike, "2610" as {segment}_other from {segment}_pivot')

In [66]:
# Can't refer to dict members in duckdb.sql, so put each member of the dict back into a separate relation
total = renamed['total']
men = renamed['men']
women = renamed['women']

In [65]:
# Mash together all three tables into one big table
main_table = duckdb.sql("select * from total inner join men using (DGUID) inner join women using (DGUID)")

In [68]:
# Compute mode shares from the big table
bike_shares = duckdb.sql("select total_bike/total_total as overall_bike_share, men_bike/men_total as men_bike_share, women_bike/women_total as women_bike_share from main_table")

In [69]:
# The filtered data is only a few thousand rows, easy enough to work with in Excel. Export a CSV if desired.
main_table.to_csv('./output.csv')